In [116]:
import os 
import numpy as np 
import pyaudio 
import matplotlib.pyplot as plt

# from scipy.signal import spectrogram
import librosa 
import ipywidgets as widgets
from IPython.display import display, Audio


import sounddevice as sd
import wave


from scipy.signal import find_peaks
import hashlib

import json
import numpy as np

In [117]:
n_fft = 2048
hop_length = n_fft // 4 
sr =  22050


# Define the ROI size
ROI_TIME = 0.5  # in seconds
ROI_FREQ = 100  # in Hz

In [118]:
def spectrogram(y): 

    n_fft = 2048 #  Number of samples in each FFT window. Higher values improve frequency resolution but reduce time resolution
    hop_length = n_fft // 4  # 512, The number of samples between successive FFT frames. Smaller values increase overlap, providing smoother time representation.

    stft = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
    spectrogram = np.abs(stft)
    spectrogram_dB = librosa.amplitude_to_db(spectrogram, ref=np.max)

    return spectrogram_dB

In [119]:
def find_peaks_spectrogram(spectrogram_dB): 

    peaks = []
    for t in range(spectrogram_dB.shape[1]):  # Loop over time frames
        
        # Apply the threshold in dB, adjusting the factor if necessary
        # threshold_dB = 0.5 * np.max(spectrogram_dB[:, t])  # threshold in dB
        threshold_dB =  0.2*np.min(spectrogram_dB[:, t])  # threshold in dB
        freq_peaks, _ = find_peaks(spectrogram_dB[:, t], height=threshold_dB)  # Apply threshold to dB values

        for f in freq_peaks:
            frequency = f * sr / n_fft  # Actual frequency in Hz
            time_point = t * hop_length / sr  # Time in seconds
            peaks.append((time_point, frequency))  # Store (time_in_seconds, frequency_bin)


    peaks = np.array(peaks)
    return peaks 

In [120]:
# Function to generate hash
def generate_hash(fa, ta, fk, tk):
    hash_input = f"{fa}-{fk}-{tk-ta}".encode('utf-8')
    return hashlib.sha1(hash_input).hexdigest()

# Select anchor points based on the highest amplitude within a certain region
def select_anchor_points(peaks, spectrogram_dB, num_anchors=5):
    anchor_points = []
    for t in range(0, spectrogram_dB.shape[1], int(ROI_TIME * sr / hop_length)):
        for f in range(0, spectrogram_dB.shape[0], int(ROI_FREQ * n_fft / sr)):

            region_peaks = peaks[
                (peaks[:, 0] >= t * hop_length / sr) & (peaks[:, 0] < (t + int(ROI_TIME * sr / hop_length)) * hop_length / sr) &
                (peaks[:, 1] >= f * sr / n_fft) & (peaks[:, 1] < (f + int(ROI_FREQ * n_fft / sr)) * sr / n_fft)
            ]
            
            if len(region_peaks) > 0:
                max_peak = region_peaks[np.argmax(spectrogram_dB[region_peaks[:, 1].astype(int) * n_fft // sr, region_peaks[:, 0].astype(int) * sr // hop_length])]
                anchor_points.append(max_peak)
    return np.array(anchor_points)

In [121]:
def get_hashes(anchor_points, peaks): 

    # List to store the hashes
    hashes = []

    # Iterate over each anchor point
    for anchor in anchor_points:
        ta, fa = anchor

        # Define the ROI
        roi_peaks = peaks[(peaks[:, 0] >= ta) & (peaks[:, 0] <= ta + ROI_TIME) & (peaks[:, 1] >= fa - ROI_FREQ) & (peaks[:, 1] <= fa + ROI_FREQ)]
        
        # Generate hashes for keypoints in the ROI
        for keypoint in roi_peaks:
            tk, fk = keypoint
            if (tk, fk) != (ta, fa):  # Exclude the anchor point itself
                hash_value = generate_hash(fa, ta, fk, tk)
                hashes.append(((fa, fk, tk - ta), ta, hash_value))

    return hashes


In [122]:
def generate_hash_audio_file(file_path):

    y_full, _ = librosa.load(file_path, sr=sr)
    spectrogram_dB_full = spectrogram(y_full)
    peaks_full= find_peaks_spectrogram(spectrogram_dB_full)
    anchor_points_full = select_anchor_points(peaks_full, spectrogram_dB_full)
    hashes_full = get_hashes(anchor_points_full, peaks_full)

    return list(map(list, hashes_full))  # Ensure the hashes are serializable


def generate_hash_audio(y_full):

    spectrogram_dB_full = spectrogram(y_full)
    peaks_full= find_peaks_spectrogram(spectrogram_dB_full)
    anchor_points_full = select_anchor_points(peaks_full, spectrogram_dB_full)
    hashes_full = get_hashes(anchor_points_full, peaks_full)

    return list(map(list, hashes_full))  # Ensure the hashes are serializable

In [123]:
def generate_hashes_for_database(database_folder):
    song_hashes = {}
    
    for song_file in os.listdir(database_folder):
        if song_file.endswith('.mp3'):
            audio_file_path = os.path.join(database_folder, song_file)

            print('generating hashes for : ', audio_file_path)
            hashes = generate_hash_audio_file(audio_file_path)

            print('number of hashes generated for : ', audio_file_path, 'is : ', len(hashes))
            print('saving in dictionary with key = ', song_file)
            song_hashes[song_file] = hashes

    return song_hashes



In [125]:
def find_best_match(hashes): 

    with open('song_hashes.json', 'r') as f:
        song_hashes = json.load(f)

    # Extract the hash values from the full song hashes 
    hash_values_full = set(h[2] for h in hashes)

    # Find the song with the most matches
    best_match_song = None
    max_matches = 0

    for song, hashes in song_hashes.items():
        hash_values = set(h[2] for h in hashes)
        common_hashes = hash_values_full.intersection(hash_values)
        num_matches = len(common_hashes)

        print('current matches : ', num_matches)
        
        if num_matches > max_matches:
            max_matches = num_matches
            best_match_song = song



    print(f"The song with the most matches is: {best_match_song} with {max_matches} matches")

    return best_match_song, max_matches

In [137]:
def find_best_match2(hashes): 
    with open('song_hashes.json', 'r') as f:
        song_hashes = json.load(f)

    # Extract the hash values from the full song hashes 
    hash_values_full = {h[2] for h in hashes}

    # Find the song with the most matches
    best_match_song = None
    best_match_score = -1  # Initialize the best match score

    for song, song_hashes_list in song_hashes.items():
        hash_values = {h2[2] for h2 in song_hashes_list}
        common_hashes = hash_values_full.intersection(hash_values)
        num_matches = len(common_hashes)
        
        # Calculate the total difference for the common hashes
        differences = []  # List to store all differences for variance calculation
        valid_matches = 0  # Count how many matches are valid

        for h in hashes:  # Query hashes
            for h2 in song_hashes_list:  # Song hashes
                if h[2] == h2[2]:  # Check if the hash value matches
                    # Ensure both hashes have at least 4 elements (the required parts of the hash)
                    if len(h) > 2 and len(h2) > 2:  # Check that both h and h2 have at least 4 elements
                        diff = abs(h[1] - h2[1])  # Calculate the difference in the second value (e.g., h[1])
                        differences.append(diff)
                        valid_matches += 1
                    else:
                        print(f"Skipping hash pair with missing h[3]: {h}, {h2}")

        # If valid matches exist, adjust the score
        if valid_matches > 0:
            # Calculate variance of the differences
            variance = sum((x - (sum(differences) / valid_matches)) ** 2 for x in differences) / valid_matches
            match_score = num_matches - variance  # Adjust the score based on variance
        else:
            match_score = num_matches  # If no valid matches, just count the hash matches

        print(f"Current song: {song}, Matches: {num_matches}, Variance: {variance if valid_matches > 0 else 0}, Score: {match_score}")

        if match_score > best_match_score:
            best_match_score = match_score
            best_match_song = song

    print(f"The song with the most matches is: {best_match_song} with {num_matches} matches")

    return best_match_song, num_matches

In [127]:
def find_best_match3(hashes): 
    with open('song_hashes.json', 'r') as f:
        song_hashes = json.load(f)

    # Find the song with the most matches
    best_match_song = None
    best_match_score = -1  # Initialize the best match score

    for song, song_hashes_list in song_hashes.items():
        num_matches = 0
        differences = []  # List to store all differences

        for h in hashes:  # Query hashes
            for h2 in song_hashes_list:  # Song hashes
                if h[2] == h2[2]:  # Check if the hash value matches
                    # Ensure both h and h2 have at least 3 elements (the required parts of the hash)
                    if len(h) > 2 and len(h2) > 2:  # Ensure h and h2 have at least 3 elements
                        diff = abs(h[1] - h2[1])  # Calculate the difference
                        differences.append(diff)
                        num_matches += 1
                    else:
                        # print(f"Skipping hash pair with missing elements: {h}, {h2}")
                        continue

        if num_matches > 0:
            # Calculate variance of the differences
            variance = np.var(differences)
            # A lower variance indicates more consistency in the matches
            match_score = num_matches - variance  # Adjust the score based on variance
        else:
            match_score = 0  # No matches found

        print(f"Current song: {song}, Matches: {num_matches}, Variance: {variance if num_matches > 0 else 0}, Score: {match_score}")

        if match_score > best_match_score:
            best_match_score = match_score
            best_match_song = song

    print(f"The song with the most matches is: {best_match_song} with {num_matches} matches")

    return best_match_song, num_matches

In [128]:
def find_best_match4(hashes): 
    with open('song_hashes.json', 'r') as f:
        song_hashes = json.load(f)

    # Find the song with the most matches
    best_match_song = None
    max_matches = 0
    best_match_score = -1  # Initialize the best match score

    for song, song_hashes_list in song_hashes.items():
        num_matches = 0
        differences = []  # List to store all differences

        for h in hashes:  # Query hashes
            for h2 in song_hashes_list:  # Song hashes
                if h[2] == h2[2]:  # Check if the hash value matches
                    # Ensure both h and h2 have at least 3 elements (the required parts of the hash)
                    if len(h) > 2 and len(h2) > 2:  # Ensure h and h2 have at least 3 elements
                        # Compare the second elements of the hashes (h[1] and h2[1])
                        diff = abs(h[1] - h2[1])  # Difference between query and song hash
                        differences.append(diff)
                        num_matches += 1
                    else:
                        print(f"Skipping hash pair with missing elements: {h}, {h2}")

        if num_matches > 0:
            # Calculate variance of the differences
            variance = np.var(differences)
            # A lower variance indicates more consistency in the matches
            match_score = num_matches - variance  # Adjust the score based on the variance
        else:
            match_score = 0  # No matches found

        print(f"Current song: {song}, Matches: {num_matches}, Variance: {variance if num_matches > 0 else 0}, Score: {match_score}")

        if match_score > best_match_score:
            best_match_score = match_score
            best_match_song = song

    print(f"The song with the most matches is: {best_match_song} with {max_matches} matches")

    return best_match_song, max_matches

In [139]:
# generating database 

# Save the hashes to a JSON file
song_hashes = generate_hashes_for_database('database')
with open('song_hashes.json', 'w') as f:
    json.dump(song_hashes, f, indent=4)

print("Hashes generated and saved to song_hashes.json")


generating hashes for :  database\01 Boss Bitch.mp3
number of hashes generated for :  database\01 Boss Bitch.mp3 is :  10105
saving in dictionary with key =  01 Boss Bitch.mp3
generating hashes for :  database\bob-sinclar-world-hold-on-official-video (1).mp3
number of hashes generated for :  database\bob-sinclar-world-hold-on-official-video (1).mp3 is :  34726
saving in dictionary with key =  bob-sinclar-world-hold-on-official-video (1).mp3
generating hashes for :  database\goodbye my lover.mp3
number of hashes generated for :  database\goodbye my lover.mp3 is :  40696
saving in dictionary with key =  goodbye my lover.mp3
generating hashes for :  database\It's Going Down Now.mp3
number of hashes generated for :  database\It's Going Down Now.mp3 is :  51601
saving in dictionary with key =  It's Going Down Now.mp3
generating hashes for :  database\ymca.mp3
number of hashes generated for :  database\ymca.mp3 is :  32584
saving in dictionary with key =  ymca.mp3
Hashes generated and saved 

## testing

In [72]:
DURATION = 20  # Duration in seconds

# Widget for starting and stopping recording
record_button = widgets.Button(description="Record")
play_button = widgets.Button(description="Play")
status_label = widgets.Label(value="Click 'Record' to start recording.")

# Variable to store the recorded audio
recorded_audio = None

def record_audio(change):
    global recorded_audio
    status_label.value = "Recording..."
    try:
        # Record audio
        recorded_audio = sd.rec(int(DURATION * sr), samplerate=sr, channels=1, dtype=np.float32)
        sd.wait()  # Wait until recording is finished
        status_label.value = "Recording complete! Click 'Play' to listen."
    except Exception as e:
        status_label.value = f"Error: {str(e)}"

def play_audio(change):
    if recorded_audio is not None:
        # Play the recorded audio
        sd.play(recorded_audio, sr)
        sd.wait()  # Wait until playback is finished
    else:
        status_label.value = "No audio recorded yet. Please record first."
        print('type of audio: ', type(record_audio))

# Link buttons to their functions
record_button.on_click(record_audio)
play_button.on_click(play_audio)


# Display widgets
display(record_button, play_button, status_label)

Button(description='Record', style=ButtonStyle())

Button(description='Play', style=ButtonStyle())

Label(value="Click 'Record' to start recording.")

In [141]:
hashes = generate_hash_audio(recorded_audio.ravel())
bestmatch, num_matches = find_best_match(hashes)

current matches :  4
current matches :  6
current matches :  22
current matches :  12
current matches :  9
The song with the most matches is: goodbye my lover.mp3 with 22 matches


In [142]:
# bestmatch, num_matches = find_best_match2(hashes)

Current song: 01 Boss Bitch.mp3, Matches: 4, Variance: 792.880791647183, Score: -788.880791647183
Current song: bob-sinclar-world-hold-on-official-video (1).mp3, Matches: 6, Variance: 1122.5532568422307, Score: -1116.5532568422307
Current song: goodbye my lover.mp3, Matches: 22, Variance: 2251.3264743771747, Score: -2229.3264743771747
Current song: It's Going Down Now.mp3, Matches: 12, Variance: 1035.0427038602252, Score: -1023.0427038602252
Current song: ymca.mp3, Matches: 9, Variance: 1220.8564043579372, Score: -1211.8564043579372
The song with the most matches is: None with 9 matches


In [143]:
# bestmatch, num_matches = find_best_match3(hashes)

Current song: 01 Boss Bitch.mp3, Matches: 0, Avg Diff: 0, Score: 0
Current song: bob-sinclar-world-hold-on-official-video (1).mp3, Matches: 0, Avg Diff: 0, Score: 0
Current song: goodbye my lover.mp3, Matches: 1, Avg Diff: 0.023219954648524777, Score: 0.9767800453514752
Current song: It's Going Down Now.mp3, Matches: 0, Avg Diff: 0, Score: 0
Current song: ymca.mp3, Matches: 0, Avg Diff: 0, Score: 0
The song with the most matches is: goodbye my lover.mp3 with 0 matches


In [144]:
# bestmatch, num_matches = find_best_match4(hashes)

Current song: 01 Boss Bitch.mp3, Matches: 9, Variance: 792.880791647183, Score: -783.880791647183
Current song: bob-sinclar-world-hold-on-official-video (1).mp3, Matches: 44, Variance: 1122.5532568422304, Score: -1078.5532568422304
Current song: goodbye my lover.mp3, Matches: 62, Variance: 2251.326474377175, Score: -2189.326474377175
Current song: It's Going Down Now.mp3, Matches: 38, Variance: 1035.0427038602252, Score: -997.0427038602252
Current song: ymca.mp3, Matches: 20, Variance: 1220.856404357937, Score: -1200.856404357937
The song with the most matches is: None with 0 matches
